In [9]:
import os
import cv2
import torch
from tqdm import tqdm
from facenet_pytorch import MTCNN

# Your dataset structure
input_dirs = {
    "real": "Dataset/real",
    "fake": "Dataset/fake"
}
output_dir = "ProcessedFrames"
os.makedirs(output_dir, exist_ok=True)

# Set device and load MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=False, device=device)

# Face extraction function
def extract_faces_from_video(video_path, output_path, sample_every_seconds=15):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second of the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idx = 0
    saved = 0
    sample_interval = fps * sample_every_seconds  # Frames to skip to get every 15 seconds

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Extract frame every `sample_interval` frames
        if frame_idx % sample_interval == 0:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            boxes, _ = mtcnn.detect(rgb)

            if boxes is not None:
                for box in boxes[:1]:  # Only first face per frame
                    x1, y1, x2, y2 = map(int, box)
                    face = frame[y1:y2, x1:x2]

                    if face.size != 0:
                        os.makedirs(output_path, exist_ok=True)
                        frame_name = f"frame_{saved:03d}.jpg"
                        cv2.imwrite(os.path.join(output_path, frame_name), face)
                        saved += 1
                        break

        frame_idx += 1
    cap.release()

# Count total frames for global progress bar
total_frames_to_process = 0
for label, folder in input_dirs.items():
    videos = [f for f in os.listdir(folder) if f.endswith(".mp4")]
    for video in videos:
        cap = cv2.VideoCapture(os.path.join(folder, video))
        total_frames_to_process += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()

# Global progress bar
with tqdm(total=total_frames_to_process, desc="Processing All Videos", unit="frame") as pbar:
    for label, folder in input_dirs.items():
        videos = [f for f in os.listdir(folder) if f.endswith(".mp4")]

        for video in videos:
            video_path = os.path.join(folder, video)
            vid_name = os.path.splitext(video)[0]
            save_folder = os.path.join(output_dir, label, vid_name)

            # Skip if already processed
            if os.path.exists(save_folder) and len(os.listdir(save_folder)) >= 1:
                pbar.update(int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT)))
                continue

            # Extract faces
            extract_faces_from_video(video_path, save_folder, sample_every_seconds=15)
            pbar.update(int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT)))

Processing All Videos: 100%|██████████| 42021/42021 [00:00<00:00, 66881.42frame/s]


In [10]:
import os

def verify_processed_frames(processed_dir="ProcessedFrames"):
    for label in ["real", "fake"]:
        label_dir = os.path.join(processed_dir, label)
        for video in os.listdir(label_dir):
            video_dir = os.path.join(label_dir, video)
            if os.path.isdir(video_dir):
                num_frames = len([f for f in os.listdir(video_dir) if f.endswith('.jpg')])
                print(f"Video {video} has {num_frames} frames.")
            else:
                print(f"Video {video} has no frames.")
                
verify_processed_frames()

Video 01__hugging_happy has 3 frames.
Video 04__talking_against_wall has 3 frames.
Video 06__hugging_happy has 3 frames.
Video 02__walking_outside_cafe_disgusted has 2 frames.
Video 06__walk_down_hall_angry has 2 frames.
Video 10__walking_outside_cafe_disgusted has 3 frames.
Video 02__hugging_happy has 3 frames.
Video 01__kitchen_pan has 2 frames.
Video 01__meeting_serious has 3 frames.
Video 09__podium_speech_happy has 3 frames.
Video 11__outside_talking_still_laughing has 3 frames.
Video 11__talking_angry_couch has 5 frames.
Video 10__kitchen_pan has 2 frames.
Video 02__outside_talking_still_laughing has 3 frames.
Video 08__walking_outside_cafe_disgusted has 2 frames.
Video 01__walking_and_outside_surprised has 4 frames.
Video 07__walking_down_street_outside_angry has 3 frames.
Video 01__exit_phone_room has 1 frames.
Video 05__outside_talking_pan_laughing has 2 frames.
Video 03__secret_conversation has 1 frames.
Video 01__podium_speech_happy has 3 frames.
Video 04__outside_talking_pa

In [11]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

def preprocess_faces(processed_dir="ProcessedFrames", target_size=(224, 224)):
    X = []
    y = []

    for label in ["real", "fake"]:
        label_dir = os.path.join(processed_dir, label)
        label_class = 0 if label == "real" else 1  # Assign label 0 for real, 1 for fake
        
        for video in os.listdir(label_dir):
            video_dir = os.path.join(label_dir, video)
            if os.path.isdir(video_dir):
                for frame_file in os.listdir(video_dir):
                    if frame_file.endswith('.jpg'):
                        frame_path = os.path.join(video_dir, frame_file)
                        # Read image, resize and normalize
                        frame = cv2.imread(frame_path)
                        frame = cv2.resize(frame, target_size)
                        frame = frame.astype("float32") / 255.0  # Normalize

                        X.append(frame)
                        y.append(label_class)

    X = np.array(X)
    y = np.array(y)

    return X, y

X, y = preprocess_faces()
print("Processed data shape:", X.shape)

Processed data shape: (137, 224, 224, 3)


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_data(X_train):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )
    datagen.fit(X_train)
    return datagen

# Apply augmentation to the dataset
datagen = augment_data(X)

In [13]:
import os

# Define the paths for the real and fake frames
real_frames_dir = "ProcessedFrames/real"
fake_frames_dir = "ProcessedFrames/fake"

# Function to count frames in a video folder
def count_frames_in_video_folder(video_folder_path):
    # Count the number of frames (files with .jpg extension)
    frames = [f for f in os.listdir(video_folder_path) if f.endswith('.jpg')]
    return len(frames)

# Function to count frames in the entire folder (real or fake)
def count_frames_in_folder(folder_path):
    total_frames = 0
    # Loop through each video folder
    for video_folder in os.listdir(folder_path):
        video_folder_path = os.path.join(folder_path, video_folder)
        if os.path.isdir(video_folder_path):
            # Count frames in the current video folder
            total_frames += count_frames_in_video_folder(video_folder_path)
    return total_frames

# Count frames in the 'real' and 'fake' directories
real_frames = count_frames_in_folder(real_frames_dir)
fake_frames = count_frames_in_folder(fake_frames_dir)

# Calculate total frames
total_frames = real_frames + fake_frames

# Print the total frames
print(f"Total frames in the 'real' directory: {real_frames}")
print(f"Total frames in the 'fake' directory: {fake_frames}")
print(f"Total frames in the dataset: {total_frames}")

Total frames in the 'real' directory: 67
Total frames in the 'fake' directory: 70
Total frames in the dataset: 137
